In [8]:
import os
import csv
 
from PIL import Image

In [3]:
!pip install imgaug
# !pip install -U scipy==1.2.0
# !pip install imageio

  Using cached imgaug-0.4.0-py2.py3-none-any.whl (948 kB)
  Using cached shapely-2.0.1-cp37-cp37m-win_amd64.whl (1.4 MB)
     ---------------------------------------- 12.1/12.1 MB 8.7 MB/s eta 0:00:00
     ---------------------------------------- 3.4/3.4 MB 9.0 MB/s eta 0:00:00
     ------------------------------------- 178.9/178.9 kB 11.2 MB/s eta 0:00:00
     ---------------------------------------- 4.2/4.2 MB 8.0 MB/s eta 0:00:00
     ---------------------------------------- 1.9/1.9 MB 9.4 MB/s eta 0:00:00


In [1]:
import os
from glob import glob
from datetime import datetime
from shutil import copyfile
 
import imgaug as ia
from imgaug import augmenters as iaa
from imageio import imread
import imageio
 
INPUT = 'splitx'
OUTPUT = 'augx'
WHITE_LIST_FORMAT = ('png', 'jpg', 'jpeg', 'bmp', 'ppm', 'JPG')
ITERATIONS = 5
# labels=['hello','thanks','yes','sorry','iloveyou']
 
def check_dir_or_create(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
 
# Sometimes(0.5, ...) applies the given augmenter in 50% of all cases,
# e.g. Sometimes(0.5, GaussianBlur(0.3)) would blur roughly every second image.
sometimes = lambda aug: iaa.Sometimes(0.5, aug)
 
# Define our sequence of augmentation steps that will be applied to every image
# All augmenters with per_channel=0.5 will sample one value _per image_
# in 50% of all cases. In all other cases they will sample new values
# _per channel_.
 
augmenters = [
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    # Strengthen or weaken the contrast in each image.
    iaa.ContrastNormalization((0.75, 1.5)),
    # Make some images brighter and some darker.
    # In 20% of all cases, we sample the multiplier once per channel,
    # which can end up changing the color of the images.
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    # Apply affine transformations to each image.
    # Scale/zoom them, translate/move them, rotate them and shear them.
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
#         translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
#         rotate=(-25, 25),
        shear=(-8, 8)
    )
]
 
seq = iaa.Sequential(augmenters, random_order=True)
 
files = [y for x in os.walk(INPUT)
         for y in glob(os.path.join(x[0], '*')) if os.path.isfile(y)]
files = [f for f in files if f.endswith(WHITE_LIST_FORMAT)]
classes = [os.path.basename(os.path.dirname(x)) for x in files]
classes_set = set(classes)
for _class in classes_set:
    _dir = os.path.join(OUTPUT, _class)
    check_dir_or_create(_dir)
 
batches = []
BATCH_SIZE = 50
batches_count = len(files) // BATCH_SIZE + 1
for i in range(batches_count):
    batches.append(files[BATCH_SIZE * i:BATCH_SIZE * (i + 1)])
 
images = []
for i in range(ITERATIONS):
# for i in labels:
    print(i, datetime.time(datetime.now()))
    for batch in batches:
        images = []
        for file in batch:
            img = imread(file)
            images.append(img)
        images_aug = seq.augment_images(images)
        for file, image_aug in zip(batch, images_aug):
            root, ext = os.path.splitext(file)
            new_filename = root + '_{}'.format(i) + ext
            new_path = new_filename.replace(INPUT, OUTPUT, 1)
                        
            imageio.imwrite(new_path, image_aug)
 
for file in files:
    dst = file.replace(INPUT, OUTPUT)
    copyfile(file, dst)

0 21:41:48.179255
1 21:41:48.262011
2 21:41:48.275973
3 21:41:48.287941
4 21:41:48.299909


C:\Users\SADNYANI\anaconda3\envs\sadnyani\lib\site-packages\imgaug\imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)
C:\Users\SADNYANI\anaconda3\envs\sadnyani\lib\site-packages\ipykernel_launcher.py:73: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.


In [2]:
# import os
# import shutil

# images = [f for f in os.listdir()]

# os.mkdir('train')

# for image in images:
#     new_path = 'downloaded_images/' + image
#     shutil.move(image, new_path)